In [20]:
# Import necessary modules in ascending order by line length
# Just b/c
import os
import time
import math
import random
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
sourceFile = 'Source.xlsx'
# Create a DataFrame of the sentences to be translated
source = pd.read_excel(sourceFile, header = None)
source.columns = ['origin', 'language', 'extra']
# Get the project name
project = source.language[1]
# Get the language
language = source.language[0]
# Drop the language column
source = source.drop(columns = 'language')
# Drop the extra column
source = source.drop(columns = 'extra')
# Drop nulls
source = source.dropna(axis = 0)
# Remove \n
source['origin'] = source['origin'].replace(r'\\n', ' ', regex = True)

## The Functions

In [21]:
# Define time functions
def fConvertMilis(seconds):
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%dh %02dm %02ds" % (hour, minutes, seconds)
# Define the scraping function
def fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot):
    fStartTime = time.time()
    print("---------{}.) {}--------".format(serviceNumber, service))
    # Set the Driver up
    # Instantiate an Options object
    option = webdriver.ChromeOptions()
    #Remove navigator.webdriver flag
    option.add_argument('--disable-blink-features=AutomationControlled')
    # Change the resolution of the browser
    option.add_argument("window-size=1920,1080")
    # Adjusting the user agent
    option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
    #Open Browser
    driver = webdriver.Chrome(executable_path='chromedriver.exe', options=option)
    # Open the specfied URL
    driver.get(url)
    # Sleep to avoid errors
    time.sleep(random.uniform(2.0, 4.0))
    # Close the popup
    if pPopup:
        bClosePopUp = driver.find_element_by_xpath(pPopup).click()
    # Open the "Select Language" dropdown menu
    if ddSelectLanguage: 
        ddSource = driver.find_element_by_xpath(ddSelectLanguage).click()
    # Sleep to avoid errors
    time.sleep(random.uniform(2.0, 4.0))
    # Click the English button
    if ddbEnglish:
        button = driver.find_element_by_xpath(ddbEnglish).click()
    # Sleep to avoid errors
    time.sleep(random.uniform(2.0, 4.0))
    # Find the text field
    textFieldSource = driver.find_element_by_xpath(tSourceInput)
    # Sleep to avoid errors
    time.sleep(random.uniform(2.0, 4.0))
    # Handle SoGous bot check
    if bRobot:
        # Type the dummy text into the text field to trigger the bot detection
        textFieldSource.send_keys('我喜欢吃冰淇淋.')
        # Sleep to allow for manual bot-detectiong passing
        time.sleep(30)
        # Sleep to avoid errors
        time.sleep(random.uniform(2.0, 4.0))
        # Find the text field and clear it
        textFieldSource = driver.find_element_by_xpath(tSourceInput).clear()
        # Open the "Select Language" dropdown menu
        buttonSource = driver.find_element_by_xpath(ddSelectLanguage).click()
        # Sleep to avoid errors
        time.sleep(random.uniform(2.0, 4.0))
        # Click the English button
        button = driver.find_element_by_xpath(ddbEnglish).click()
        # Find the text field once more
        textFieldSource = driver.find_element_by_xpath(tSourceInput)
    # Type the text into the text field
    batch = 1
    sentencesCleanBatches = {}
    sentencesClean = []
    nLimit = 4000
    nDeepLimit = 1000
    nQQLimit = 1800
    # Get number of rows
    sTotalRows = source.shape[0]
    for index, row in source.iterrows():
        # Print to console some info
        #if :
            #print("Row {} of {}".format(index + 1, sTotalRows))
            #print("processing...")
        oIndex = index
        oNumChars = source['origin'][0:index].str.len().sum()
        # Input the row's text
        textFieldSource = driver.find_element_by_xpath(tSourceInput).send_keys(source['origin'][index])
        textFieldSource = driver.find_element_by_xpath(tSourceInput).send_keys(Keys.RETURN)  
        # Check to see if the site is DeepL     
        if 'deepl' in url and oNumChars >= nDeepLimit:
            # Pause to let the translations load
            time.sleep(7)
            # Increment the limit for the next batch
            nDeepLimit += 1000
        if 'qq' in url and oNumChars >= nQQLimit:
            # Increment the limit for the next batch
            nQQLimit += 1800
            # Pause to avoid errors
            time.sleep(7)
            # Store the translated text as a string
            if "text" in oExtractMethod:
                dTranslatedText = driver.find_element_by_xpath(tTranslatedText).text
            else:
                dTranslatedText = driver.find_element_by_xpath(tTranslatedText).get_attribute('innerHTML').strip()
            # Sleep to avoid errors
            time.sleep(2)
            # Clean the output text and print
            sentencesCleanBatches[batch] = dTranslatedText.split("\n")
            # Print the batch number and its length        
            print(f"Batch {batch}: {len(sentencesCleanBatches[batch])}")
            batch += 1
            # Find the text field once more and clear it
            if 'qq' or 'sogou' in url:
                driver.refresh()
            else:
                textFieldSource = driver.find_element_by_xpath(tSourceInput).clear()
        # Check to see if the number of chars is too much
        if oNumChars > nLimit:
            # Increment the limit for the next batch
            nLimit += 4000
            # Pause to avoid errors
            time.sleep(7)
            # Store the translated text as a string
            if "text" in oExtractMethod:
                dTranslatedText = driver.find_element_by_xpath(tTranslatedText).text
            else:
                dTranslatedText = driver.find_element_by_xpath(tTranslatedText).get_attribute('innerHTML').strip()
            # Sleep to avoid errors
            time.sleep(3)
            # Clean the output text and print
            sentencesCleanBatches[batch] = dTranslatedText.split("\n")
            # Print the batch number and its length        
            print(f"Batch {batch}: {len(sentencesCleanBatches[batch])}")
            batch += 1
            # Refresh the driver for the next batch
            if 'google' in url:
                textFieldSource = driver.find_element_by_xpath(tSourceInput).clear()
            else:
                driver.refresh()
    # Pause to avoid errors
    time.sleep(7)
    # Store the translated text as a string
    if "text" in oExtractMethod:
        dTranslatedText = driver.find_element_by_xpath(tTranslatedText).text
    else:
        dTranslatedText = driver.find_element_by_xpath(tTranslatedText).get_attribute('innerHTML').strip()
    # Sleep to avoid errors
    time.sleep(3)
    # Clean the output for the last batch
    sentencesCleanBatches[batch] = dTranslatedText.split("\n")
    # Print the batch number and its length        
    print(f"Batch {batch}: {len(sentencesCleanBatches[batch])}")
    # Concatenate all the batches into once list
    for i in range(1, batch + 1):
        dSentencesClean[serviceNumber] = sentencesClean + sentencesCleanBatches[i]
    # Sleep to avoid errors
    time.sleep(2)
    # Remove period strings in the list
    noPeriods = dSentencesClean[serviceNumber]
    while("." in noPeriods): 
        noPeriods.remove(".")
    dNoPeriods[serviceNumber] = noPeriods
    # Strip extra things in the list
    stripped = noPeriods
    stripped = [item.strip() for item in stripped]
    dStripped[serviceNumber] = stripped
    # Remove empty strings in the list
    noEmpty = stripped
    while("" in noEmpty): 
        noEmpty.remove("")
    dNoEmpty[serviceNumber] = noEmpty
    # Define the dictionary DF
    dSentencesCleanFinal[serviceNumber] = dNoEmpty[serviceNumber]
    # Define the end time
    fEndTime = time.time()
    # Print to console for the report
    print("Rows: {}".format(sTotalRows))
    print("Duration: {}".format(fConvertMilis(fEndTime - fStartTime)))
    print("Length: {}".format(len(dSentencesCleanFinal[serviceNumber])))
    # Close the driver
    driver.close()
    # Return said DF
    return dSentencesCleanFinal[serviceNumber], sTotalRows, dNoPeriods[serviceNumber], dStripped[serviceNumber], dNoEmpty[serviceNumber], dSentencesClean[serviceNumber]

## The Program

In [22]:
# Define start time
oStartTime = time.time()
# Define service number to start with
serviceNumber = 0
# Create a dictionary to store the lists
dSentencesClean = {}
dNoPeriods = {}
dStripped = {}
dNoEmpty = {}
dSentencesCleanFinal = {}
# Create a list to store the service names
lServices = ['source']
# Define the source column
dSentencesCleanFinal[0] = source['origin']

# Baidu
serviceNumber += 1 
service = 'Baidu'
url = 'https://fanyi.baidu.com/#zh/en/'
ddSelectLanguage = False
ddbEnglish = False
tSourceInput = '//*[@id="baidu_translate_input"]'
tTranslatedText = '//*[@id="main-outer"]/div/div/div[1]/div[2]/div[1]/div[2]/div/div/div[1]'
pPopup = '/html/body/div[1]/div[7]/div/div/a[2]'
oExtractMethod = "text"
bRobot = False
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

# DeepL
serviceNumber += 1 
service = 'DeepL'
url = 'https://www.deepl.com/translator'
ddSelectLanguage = False
ddbEnglish = False
tSourceInput = '//*[@id="dl_translator"]/div[5]/div[2]/div[1]/div[2]/div[1]/textarea'
tTranslatedText = '//*[@id="target-dummydiv"]'
pPopup = False
oExtractMethod = "get_attribute('innerHTML').strip()"
bRobot = False
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

# Google
serviceNumber += 1 
service = 'Google'
url = 'https://translate.google.com/'
ddSelectLanguage = False
ddbEnglish = False
tSourceInput = '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[1]/span/span/div/textarea'
tTranslatedText = '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[2]/div[5]/div/div[1]'
pPopup = False
oExtractMethod = "text"
bRobot = False
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

# SoGou
serviceNumber += 1 
service = 'SoGou'
url = 'https://translate.sogou.com/text'
ddSelectLanguage = '//*[@id="J-langselect"]/div/span[3]'
ddbEnglish = '/html/body/div/div/div[2]/div[1]/div/div[2]/div[4]/div[1]/span[2]'
tSourceInput = '//*[@id="trans-input"]'
tTranslatedText = '//*[@id="trans-result"]'
pPopup = False
oExtractMethod = "get_attribute('innerHTML').strip()"
bRobot = True
fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
lServices.append(service)

# # QQ
# serviceNumber += 1
# service = 'QQ'
# url = 'https://fanyi.qq.com/'
# ddSelectLanguage = '//*[@id="language-button-group-target"]/div[1]'
# ddbEnglish = '//*[@id="language-button-group-target"]/div[2]/ul/li[2]/span'
# tSourceInput = '/html/body/div[2]/div[2]/div[2]/div[1]/div[1]/textarea'
# tTranslatedText = '/html/body/div[2]/div[2]/div[2]/div[2]/div[2]'
# pPopup = False
# oExtractMethod = "text"
# bRobot = False
# fScrape(service, serviceNumber, url, ddSelectLanguage, ddbEnglish, tSourceInput, tTranslatedText, pPopup, oExtractMethod, bRobot)
# lServices.append(service)

# CREATE A DATAFRAME
# Define the DF
df = pd.DataFrame(columns = lServices)
# Define the index variable
x = 0
# Populate the columns
for i in lServices:
    df[i] = dSentencesCleanFinal[x] 
    x += 1
# Drop nan rows
df.dropna(how = 'all', inplace = True, axis = 0)
# Export DataFrame as an XLSX to Google Drive\Work\Translation folder
df.to_excel('{}\{}.xlsx'.format(language, project), index=False)
os.startfile('{}\{}.xlsx'.format(language, project))
# Define end time
oEndTime = time.time()
print("{} rows were scraped from {} in {}".format(nTotalRows, lServices[1:len(lServices+1)], fConvertMilis(oEndtime - oStartTime)))

---------1.) Baidu--------
Batch 1: 138
Batch 2: 127
Batch 3: 6
Rows: 271
Duration: 0h 02m 40s
Length: 6
---------2.) DeepL--------


WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.150)


## Testing

In [4]:
print(len(dSentencesClean[2]))
print(len(dNoPeriods[2]))
print(len(dStripped[2]))
print(len(dNoEmpty[2]))
print(len(dSentencesCleanFinal[2]))

6
6
6
6
6


In [5]:
dSentencesCleanFinal[0] = source['origin']
x = 0
for i in lServices:
    print("{} ({}): {}".format(i, x, len(dSentencesCleanFinal[x])))
    x += 1

source (0): 271
Baidu (1): 6
DeepL (2): 6
Google (3): 6
SoGou (4): 6


In [6]:
dSentencesCleanFinal[0] = source['origin']
x = 0
for i in lServices:
    print("{} ({}): {}".format(i, x, dSentencesCleanFinal[x]))
    x += 1

source (0): 0      开启后可获得： [00d805]随机获得1个武器或肩铠套装石[-]  打造套装所需的套装石个...
1                                               珍稀祝福晶石宝箱
2      开启后可获得： [00d805]随机获得1个武器或肩铠祝福晶石[-]  打造套装所需的祝福晶...
3                                                 符文融晶宝箱
4      使用后可获得[DAA249]1颗符文融晶[-] [75A8D9]可用来合成双属性符文，获得更...
                             ...                        
266    增强战力必备！开启可随机获得下列15阶橙色原石之一: [DAA249][地]稀有风原石[00...
267    增强战力必备！开启可随机获得下列16阶橙色原石之一: [DAA249][地]稀有风原石[00...
268    增强战力必备！开启可随机获得下列17阶橙色原石之一: [DAA249][地]稀有风原石[00...
269    增强战力必备！开启可随机获得下列18阶橙色原石之一: [DAA249][地]稀有风原石[00...
270    增强战力必备！开启可随机获得下列19阶橙色原石之一: [DAA249][地]稀有风原石[00...
Name: origin, Length: 271, dtype: object
Baidu (1): ['It is necessary to enhance the combat power! Open to randomly obtain one of the following 14 levels of orange protoliths: [daa249] [di] rare wind protolith [00d805] (+ 62443 combat power) [-] [di] rare thunder protolith [00d805] (+ 62443 combat power) [-] [di] rare water protolith [00d805] (+ 62446 combat 

In [7]:
dSentencesCleanFinal[1]

['It is necessary to enhance the combat power! Open to randomly obtain one of the following 14 levels of orange protoliths: [daa249] [di] rare wind protolith [00d805] (+ 62443 combat power) [-] [di] rare thunder protolith [00d805] (+ 62443 combat power) [-] [di] rare water protolith [00d805] (+ 62446 combat power) [-] [di] rare fire protolith [00d805] (+ 62450 combat power) [-] [di] rare wood protolith [00d805] (+ 62450 combat power) [-] [di] rare gold protolith [00d805] (+ 62450 combat power) [-] [ground] rare light stone [00d805] (+ 62440 combat power) [-] [ground] rare dark stone [00d805] (+ 62440 combat power) [-] [ground] rare time stone [00d805] (+ 62450 combat power) [-] [ground] rare air stone [00d805] (+ 62450 combat power) [-]',
 'It is necessary to enhance the combat power! Open to randomly obtain one of the following 15 order orange protoliths: [daa249] [di] rare wind protolith [00d805] (+ 76806 combat power) [-] [di] rare thunder protolith [00d805] (+ 76806 combat power) [

In [270]:
project = 'zailai3_all'
# Define the DF
df = pd.DataFrame(columns = lServices)
# Populate columns
df['source'] = dSentencesCleanFinal[0]
#df['QQ'] = dSentencesCleanFinal[1]
df['Baidu'] = dSentencesCleanFinal[2]
#df['Google'] = dSentencesCleanFinal[3]
df['DeepL'] = dSentencesCleanFinal[3]
df['SoGou'] = dSentencesCleanFinal[4]
# Export DataFrame as an XLSX to Google Drive\Work\Translation folder
df.to_excel('{}\{}.xlsx'.format(language, project), index=False)
os.startfile('{}\{}.xlsx'.format(language, project))

In [267]:
df = pd.DataFrame(columns=['source', 'QQ'])
#df['source'] = dSentencesClean[0]
df['QQ'] = dSentencesCleanFinal[1]
# Export DataFrame as an XLSX to Google Drive\Work\Translation folder
df.to_excel('{}\{}.xlsx'.format(language, project), index=False)
os.startfile('{}\{}.xlsx'.format(language, project))

In [43]:
blah = []
elephant = 'elephant'
chicken[1] = ''
chicken[2] = ''


for i in range(1, 2 + 1):
    blah.append(chicken[i])